### Imports

In [ ]:
def LinearRegressionBaseLine(X_train,X_test,Y_train,Y_test):
    
    #Building model with Train data
    reg_model_built_on_train= LinearRegression().fit(X_train, Y_train)
    print("Model details are coefficients of "+str(reg_model_built_on_train.coef_)+" interecpt of "+str(reg_model_built_on_train.intercept_))

    Y_val_pred = reg_model_built_on_train.predict(X_val)
    rmse = root_mean_squared_error(Y_val,Y_val_pred)
    r2 = r2_score(Y_val,Y_val_pred)

    print("For model built on train data and predictions compared on val data : Model has rmse of "+ str(rmse)+" and r2 of "+str(r2))

    #Test the model 
    Y_test_pred= reg_model_built_on_train.predict(X_test)

    rmse_for_test_data = root_mean_squared_error(Y_test,Y_test_pred)
    r2_for_test_data= r2_score(Y_test,Y_test_pred)

    print("For model when predicting on test data : Model has rmse of "+ str(rmse_for_test_data)+" and r2 of"+str(r2_for_test_data))

    return Y_test_pred


In [1]:
import pyspark
import requests
import pandas as pd

from io import StringIO
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [2]:
spark = SparkSession.builder.appName("news_data_analysis").config('spark.jars.packages', "com.google.cloud.spark:spark-3.5-bigquery:0.41.0").getOrCreate()

24/10/18 16:51:35 WARN Utils: Your hostname, Yi-Nings-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.90 instead (on interface en0)
24/10/18 16:51:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/oryining/.ivy2/cache
The jars for the packages stored in: /Users/oryining/.ivy2/jars
com.google.cloud.spark#spark-3.5-bigquery added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b4919081-4d0b-48e2-98cf-a39d6fecba83;1.0
	confs: [default]


:: loading settings :: url = jar:file:/opt/homebrew/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found com.google.cloud.spark#spark-3.5-bigquery;0.41.0 in central
	found com.google.cloud.spark#spark-bigquery-dsv2-common;0.41.0 in central
	found com.google.cloud.spark#spark-bigquery-connector-common;0.41.0 in central
	found com.google.cloud.spark#bigquery-connector-common;0.41.0 in central
	found com.google.api.grpc#grpc-google-cloud-bigquerystorage-v1;3.9.0 in central
	found io.grpc#grpc-api;1.66.0 in central
	found com.google.code.findbugs#jsr305;3.0.2 in local-m2-cache
	found com.google.errorprone#error_prone_annotations;2.23.0 in central
	found io.grpc#grpc-stub;1.66.0 in central
	found io.grpc#grpc-protobuf;1.66.0 in central
	found com.google.api.grpc#proto-google-common-protos;2.43.0 in central
	found com.google.protobuf#protobuf-java;3.25.3 in central
	found com.google.api.grpc#proto-google-cloud-bigquerystorage-v1;3.9.0 in central
	found com.google.api#api-common;2.35.0 in central
	found com.google.auto.value#auto-value-annotations;1.11.0 in central
	found javax.annotation

In [3]:
df = spark.read.format("bigquery").option("table", "gdelt-bq.gdeltv2.ggg").load()
df.show(5)

+-------------------+--------------------+--------------------+------------+--------+------------------+-----------------+--------------------+-------+-------+-----------+--------+--------+-------+--------------------+--------------------+
|           DateTime|                 URL|               Title|SharingImage|LangCode|           DocTone|DomainCountryCode|            Location|    Lat|    Lon|CountryCode|Adm1Code|Adm2Code|GeoType|      ContextualText|            the_geom|
+-------------------+--------------------+--------------------+------------+--------+------------------+-----------------+--------------------+-------+-------+-----------+--------+--------+-------+--------------------+--------------------+
|2024-10-15 06:00:00|https://www.thisd...|Stop Ignoring Nat...|        NULL|     eng|   1.2779552715655|               NI|Aborisade, Nigeri...|   7.25| 4.4167|         NI|    NI00|   23001|      4|aborisade in abuj...|  POINT(4.4167 7.25)|
|2024-10-15 06:00:00|https://www.thisd..

In [4]:
us_news = df.filter(df.CountryCode == "US")

### Baseline with FinBert

In [5]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
# UDF for BERT-based sentiment classification
def sentimentWithBert(input_text):
    import torch
    from transformers import AutoTokenizer, BertForSequenceClassification

    tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
    model = BertForSequenceClassification.from_pretrained("ProsusAI/finbert")

    inputs = tokenizer(input_text, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

    index_of_largest = torch.argmax(predictions).item()
    sentiments = ['positive', 'negative', 'neutral']
    
    sentiment = sentiments[index_of_largest]
    
    return sentiment 

# Define the UDF
classify_sentiment_udf = udf(sentimentWithBert, StringType())

df_sentiment = us_news.withColumn("Sentiment", classify_sentiment_udf(us_news["ContextualText"]))

df_sentiment.show(100, False)

+-------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+--------+------------------+-----------------+--------------------------------------------------------+-------+--------+-----------+--------+--------+-------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------